## Load libs and define some helpers

In [1]:
from PIL import Image
def show(img):
    image = Image.open(img)
    new_size = (600, 400)
    resized_image = image.resize(new_size)
    display(resized_image)

In [2]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [3]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

Let's load a Andrej K blog post about training NN (https://karpathy.github.io/2019/04/25/recipe/)

In [12]:
with open('karpathy-recipe-nn.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [13]:
text = text.replace("\n", " ")

# QA for big documents

If we want to provide context to a prompt where the information is in a large corpus of text, we are limited by:

* Limitations on the maximum number of tokens
* High processing costs
* Unnecessary processing

There is a strategy where we divide the text into smaller fragments or `chunks` and convert them into embeddings.

_An embedding is a numerical representation of an object or entity in a vector space. In the context of natural language processing, an embedding is used to represent words or phrases as vectors of real numbers. These vectors are designed in such a way that similar words or phrases in terms of meaning are close together in the vector space. Embeddings are used in tasks such as text classification, machine translation, text generation, and semantic search._

<div style="text-align:center">
<img src="./embedding.png" alt="embedding" width="600" height="300"/>
</div>

In our case, we will use the embeddings from OPENAI, specifically the `text-embedding-ada-002` model, which is very efficient for this purpose and has low costs.
To avoid reprocessing the text every time we need its embeddings, we can store them in VDB (Vector Data Bases).
In this example, we will use `Pinecone`, which is a Cloud service.
The pipeline would look like this:

<div style="text-align:center">
<img src="./docQA.png" alt="embedding" width="600" height="300"/>
</div>

In [16]:
import os
import openai
import pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.schema import Document

We create the index in Pinecone.

In dimensions, we need to provide the length of our vector, for the case of `text-embedding-ada-002`, it returns a vector of dimension 1536.

In [17]:
embeddings = OpenAIEmbeddings()

In [18]:
query_result = embeddings.embed_query("Hello world")

We can check that the embedding dim is 1536

In [19]:
len(query_result)

1536

In [20]:
import pinecone

pinecone.init(
    api_key=os.environ.get('PINECONE_KEY'),
    environment="northamerica-northeast1-gcp"
)


index_name = "blog-summary"

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(query_result),#1536
        metric='cosine',
        metadata_config={'indexed': ['channel_id', 'published']}
    )


In [21]:
from datetime import datetime
date = datetime.now()

If we use Lanchain we provide the `Document` object

In [22]:
#We create the document object mannualy in this case
documents = [Document(page_content=text,
         metadata={
             'my_document_id' : 1,
             'my_document_source' : "Jetta Rewiew",
             'my_document_create_time' : int(date.timestamp())
         })]

We proceed to perform a split. In this case, we divide the text into chunks of 1000 tokens with an overlap of 20. The chunk_overlap parameter is used to specify the number of tokens that overlap between consecutive chunks. This is useful when splitting a text to maintain continuity of context between the chunks. By including some overlapping tokens, we can ensure that a small portion of the context is shared between adjacent chunks, which can help preserve meaning and coherence when processing the text.

In [23]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)
print(len(docs))


23


In [24]:
docs[0].page_content

'A Recipe for Training Neural Networks Apr 25, 2019  Some few weeks ago I posted a tweet on “the most common neural net mistakes”, listing a few common gotchas related to training neural nets. The tweet got quite a bit more engagement than I anticipated (including a webinar :)). Clearly, a lot of people have personally encountered the large gap between “here is how a convolutional layer works” and “our convnet achieves state of the art results”.  So I thought it could be fun to brush off my dusty blog to expand my tweet to the long form that this topic deserves. However, instead of going into an enumeration of more common errors or fleshing them out, I wanted to dig a bit deeper and talk about how one can avoid making these errors altogether (or fix them very fast). The trick to doing so is to follow a certain process, which as far as I can tell is not very often documented. Let’s start with two important observations that motivate it.  1) Neural net training is a leaky abstraction It'

Now we can store the embeddings in our VDB, which will generally have the following structure:
`(ID, [embedding], metadata)`
    
    Where the ID is unique, the embedding is a vector of dimension 1536 (in our case), and the metadata can be a JSON-like object accompanying the vector. We can include the represented text as well as other attributes.

In [25]:
# connect to index
index = pinecone.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 11}},
 'total_vector_count': 11}

In [26]:
embeddings = OpenAIEmbeddings()

In [27]:
embeddings.model

'text-embedding-ada-002'

In [28]:
index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

We can estimate costs:

In [29]:
## Cost per 1000 tokens
# https://openai.com/pricing
gpt35turbo_cost = 0.002
ada_embeddings_cost = 0.0004

In [35]:
print(f'Est. cost : {sum([num_tokens_from_string(d.page_content,"cl100k_base") for d in docs]) *  0.001 * ada_embeddings_cost:.4f} $USD')

Est. cost : 0.0019 $USD


At this point, we already have our document stored in the database, and we are left with the final part.

If we want to ask a question about that document, we can make a query where:

* We need to process our question (Embedding).
* We will search in Pinecone for those chunks that are semantically similar to our question (we can request the top k nearest results).
* Once we have those results, we can create a prompt as we would normally do, but only on that specific fragment of text where the answer to our question should be found.

In [31]:
limit = 3750
index = pinecone.Index(index_name)

def retrieve(query):
    while True:
        try:
            res = openai.Embedding.create(
                input=[query],
                engine='text-embedding-ada-002'
            )
            break
        except openai.error.APIConnectionError as e:
            print(f'{e} Retrying...')
            continue

    
    # retrieve from Pinecone
    xq = res['data'][0]['embedding']
    
    

    # get relevant contexts
    res = index.query(xq, top_k=3, include_metadata=True)
    contexts = [
        x['metadata']['text'] for x in res['matches']
    ]
    

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt

We choose the 3 clostest docs to answer the query

In [32]:
query = "What is the first step in this recipe?"
query_with_contexts = retrieve(query)
query_with_contexts

'Answer the question based on the context below.\n\nContext:\nto a new problem, which I will try to describe. You will see that it takes the two principles above very seriously. In particular, it builds from simple to complex and at every step of the way we make concrete hypotheses about what will happen and then either validate them with an experiment or investigate until we find some issue. What we try to prevent very hard is the introduction of a lot of “unverified” complexity at once, which is bound to introduce bugs/misconfigurations that will take forever to find (if ever). If writing your neural net code was like training one, you’d want to use a very small learning rate and guess and then evaluate the full test set after every iteration.  1. Become one with the data The first step to training a neural net is to not touch any neural net code at all and instead begin by thoroughly inspecting your data. This step is critical. I like to spend copious amount of time (measured in uni

<br>
Finnaly we pass the prompt to `text-davinci-003` (We could use `gpt-3.5-turbo` also)

In [33]:
def complete(prompt):
    # query text-davinci-003
    res = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        temperature=0,
        max_tokens=400,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return res['choices'][0]['text'].strip()

In [34]:
complete(query_with_contexts)

'The first step is to become one with the data by thoroughly inspecting it.'

### More:

No Langchain

This is the equivalent to `Pinecone.from_documents`

In [ ]:
from tqdm.auto import tqdm
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(new_data), batch_size)):
    # find end of batch
    i_end = min(len(new_data), i+batch_size)
    meta_batch = new_data[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts, engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'start': x['start'],
        'end': x['end'],
        'title': x['title'],
        'text': x['text'],
        'url': x['url'],
        'published': x['published'],
        'channel_id': x['channel_id']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

In [85]:
chunks = []
stride = 100
window = 100

for i in range(0,len(text.split(' ')),stride):
    chunks.append(' '.join(text.split(' ')[i:window+i]))

Sources:

https://docs.pinecone.io/docs/query-data

https://platform.openai.com/docs/guides/embeddings/what-are-embeddings

https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/pinecone.html

https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb